<a href="https://colab.research.google.com/github/Friedrichz/crypto_tooling/blob/main/track_vc_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install messari

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from messari.messari import Messari
import pandas as pd
import numpy as np
import urllib.request
import json
import time
# API keys
messari_api_key = "88bbfa31-1b0c-442b-b748-cc13ed2cab1a"
messari = Messari(messari_api_key)
nomics_api_key = "91714c15b8333b59ec0806fe9fcf5179410e14b2"

In [4]:
# FOR FUNDRAISING DATA USE Messari
li_dic = [] 
for i in range(1, 5, 1):
    data = messari.get_all_assets(page=i, limit=500, to_dataframe=True)
    li_dic.append(data)

# Merge raw API data into one list with records 
data = [j for i in li_dic for j in i["data"]]
len(data)

# Inspect coins
# [(n,i) for n, i in enumerate([i["name"] for i in data])]

2000

In [5]:
# Get list of private investors
def get_investors(record):
  name = record["name"]
  symbol = record["symbol"]
  investors = []
  coin_investors = record["profile"]["investors"]["organizations"] 
  if coin_investors != None:
    investors = [i["name"] for i in coin_investors]

  return dict(
      name = name,
      symbol = symbol,
      investors = investors,
  )

inv_data = [get_investors(record) for record in data]
print(len(inv_data))
inv_df = pd.DataFrame.from_records(inv_data).drop_duplicates("symbol")
inv_df1 = inv_df.explode("investors")

2000


In [6]:
# USE NOMICS API TO GET MAX SUPPLY DATA: https://nomics.com/docs/#tag/Currencies-Ticker
# All tickers
ticker_list = [r["symbol"] for r in data]
ticker_list = [i.replace("$","").replace("-","").replace(".","").replace(" ","") for i in list(filter(None, ticker_list))]

def get_nomics_currency_data(ticker_list, parts=2, per_page=100):

  tl_parts = np.array_split(ticker_list, parts)
  data_list = []

  for tli in tl_parts: 
    # Make one long string
    tli_str = ','.join(tli)
    # 100 items per page resulting in n_pages
    n_ticks = len(tli)
    n_pages = int(np.ceil(n_ticks / per_page))

    # Get response from all pages
    for page in range(1, n_pages+1): 
      print(n_ticks, page)
      nomics_url = """https://api.nomics.com/v1/currencies/ticker?key={}&interval=1d&ids={}&per-page={}&page={}""".format(nomics_api_key, tli_str, per_page, page)
      response = urllib.request.urlopen(nomics_url).read()
      data_nomics = json.loads(response.decode('utf-8'))
      # Append new data
      data_list.append(data_nomics)
      time.sleep(1)

  # Unnest list of records
  data_list = [i for j in data_list for i in j]

  print(len(ticker_list))
  print(len(data_list))
  return data_list

# Fetch Nomics data
nomics_data = get_nomics_currency_data(ticker_list)
# Save Nomics currency data (For max supply)
nomics_df = pd.DataFrame.from_records(nomics_data)
# nomics_df.to_csv("nomics_currencies_data_supply.csv")

# Format for analysis
# nomics_df = nomics_df.set_index("symbol")
supply_cols = ['symbol','status','market_cap','circulating_supply','max_supply','first_trade']
nomics_df = nomics_df[supply_cols]

986 1
986 2
986 3
986 4
986 5
986 6
986 7
986 8
986 9
986 10
986 1
986 2
986 3
986 4
986 5
986 6
986 7
986 8
986 9
986 10
1972
1744


# Cleanup raw records 

In [7]:
def get_nested_items(list_records, attr): 
  if list_records is None:
    return []
  else: 
    return [i[attr] for i in list_records]

def clean_sales_round_title(title): 
  if title is None: pass
  else:
    # Remove digits
    title = ''.join([i for i in title if not i.isdigit()]) 
    # Lowercase & strip trailing/leading spaces
    title = title.lower().strip()
    return title

def clean_sales_rounds(list_records): 
  if list_records is None:
    pass
  else:
    [i.update(title_clean=clean_sales_round_title(i["title"])) for i in list_records]
    return

# Get list of sales_rounds labels to identify venture round / seed sale 
sales_rounds_titles = set(j for i in data for j in get_nested_items(i["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"], "title"))

# Taken manually from list of titles in data 
private_sales_rounds_titles = [
  'Fundraising ', 
  'Pre Seed Round', 
  'Pre-Sale SAFT', 
  'Private Presale', 
  'Private Sale', 
  'Private Sale1 ',
  'SAFT',
  'Seed', 
  'Seed ', 
  'Seed Funding', 
  'Seed Round', 
  'Seed Round (SAFT) ', 
  'Seed Sale', 
  'Seed Round (Crypto)', 
  'Seed Token Sale',
  'Strategic Funding Round',
  'Strategic Investments',
  'Strategic Private Sale',
  'Strategic Round',
  'Strategic Sale',
  'Token Pre-Sale',
  'Token Private Round',
  'Token Seed Round',
  'Venture Capital',
  'Venture Funding',
  'Venture Round',
]

# Clean up sales round titles & add "title_clean" to records
private_sales_rounds_titles_clean = set(clean_sales_round_title(i) for i in private_sales_rounds_titles)
print(len(private_sales_rounds_titles_clean)) # Include 24 titles that point to private token sales
_ = [clean_sales_rounds(j["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]) for j in data]

24


# Fundraise table

In [8]:
# Extract full fundraising data for each coin
def full_fundraise_data(record):
  # Returns a list of recorded funraise events 
  symbol = record["symbol"]
  name = record["name"]

  # Get details on sales rounds
  sales_rounds = record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]
  if sales_rounds:
    _ = [i.update(name=name) for i in sales_rounds]
    _ = [i.update(symbol=symbol) for i in sales_rounds]

  return sales_rounds

fundraises_list = [j for i in filter(None, [full_fundraise_data(i) for i in data]) for j in i]

# Format to df
cols = ["name", "symbol", "title_clean", "type", "end_date", "native_tokens_allocated", "equivalent_price_per_token_in_usd", "amount_collected_in_usd", "asset_collected"]
fdf = pd.DataFrame.from_records(fundraises_list)[cols]
fdf = fdf.dropna(how="all", subset=["native_tokens_allocated", "equivalent_price_per_token_in_usd", "equivalent_price_per_token_in_usd"])

# Clean "type" column
clean_dict = {
  # Private Sale
 'SAFT': 'Private', 
 'Saft': 'Private', 
 'Private Crowdsale': 'Private', 
 'Pre-Sale': 'Private', 
 'Private Sale':'Private', 
 'Private ': 'Private', 
 'private': 'Private', 
 'Accredited-only': 'Private', 
 'Seed Sale': 'Private', 
 'Seed': 'Private', 
 'Private Sale ICO': 'Private', 
 'Private (accredited investors only)': 'Private', 
 'Private (Accredited Investors Only)': 'Private', 
 'VC': 'Private', 
 'Vc': 'Private', 
 'Presale': 'Private', 
 'Pre Sale': 'Private', 
 'Pre-Sale Tranche 1': 'Private', 
 'Pre-Sale Tranche 2': 'Private', 
 'Seed Funding': 'Private', 
 'PreSale': 'Private', 
 'Seed Round': 'Private', 
 'LP': 'Private', 
 'Lp': 'Private',
 'Saft - Private, Accredited-Only': 'Private', 
 'SAFT - Private, accredited-only': 'Private', 
 'Private, accredited-only': 'Private', 
 'Private, Accredited-Only': 'Private', 
 # ICO
 'Initial Coin Offering': 'ICO',
 'Initial coin offering': 'ICO',
 'Initial Offering': 'ICO',
 'Ico': 'ICO',
 # IDO
 'Initial Dex Offering': 'IDO',
 'Initiate DEX Offering': 'IDO',
 'Initiate Dex Offering': 'IDO',
 'ido':'IDO',
 # IEO
 'Ieo': 'IEO',
 'Initial Exchange Offering':'IEO',
 'Initial Exchange Offering: Binance Smart Chain': 'IEO',
 # Public
 'Public with allocation': 'Public',
 'Public Sale - Fair Lottery': 'Public',
 'Public Sale': 'Public',
 'public': 'Public', 
 # Crowdsale
 'Crowd sale': 'Crowdsale',
 # Strategic
 'Strategic Investments': 'Strategic',
}

f = lambda x: clean_dict[x] if x in clean_dict.keys() else x
fdf["type"] = fdf["type"].map(f)
fdf["type"] = fdf["type"].str.lower()

# Add cumulative amount raised up to & incl fundraising event
fdf["cum_amount_collected_in_usd"] = fdf.sort_values("end_date", ascending=True).groupby("symbol")["amount_collected_in_usd"].cumsum()
fdf.head()

,name,symbol,title_clean,type,end_date,native_tokens_allocated,equivalent_price_per_token_in_usd,amount_collected_in_usd,asset_collected,cum_amount_collected_in_usd
0,Ethereum,ETH,token sale,public,2014-09-02T16:00:00Z,576000000.0,0.310,18300000.0,BTC,18300000.0
1,BNB,BNB,initial coin offering (ico),public,2017-07-21T04:00:00Z,100000000.0,0.150,15000000.0,"ETH, BTC",15000000.0
12,Solana,SOL,seed sale,private,2018-04-05T00:00:00Z,79290466.0,0.040,3170000.0,USD,3170000.0
13,Solana,SOL,founding sale,series a,2018-06-03T00:00:00Z,63151982.0,0.200,12630000.0,USD,15800000.0
14,Solana,SOL,validator sale,series a,2019-07-09T00:00:00Z,25331653.0,0.225,5700000.0,USD,21500000.0


# Current market data

In [9]:
# Extract (current) market data --> metrics: token_sale_stats>sale_proceeds_usd, supply_distribution
def get_market_data(record): 
  # Get current supply (see docs: https://messari.io/article/messari-proprietary-methods)
  circ_supply = record["metrics"]["supply"]["circulating"] # liquid supply that excludes project, foundation or founder units which have not been yet sold
  liquid_supply = record["metrics"]["supply"]["liquid"] # supply taht is visible on-chain and which is not known to have any programmatic or contractual restrictions

  # Get ath price data
  ath_price = record["metrics"]["all_time_high"]["price"]
  ath_date = record["metrics"]["all_time_high"]["at"]

  # Get cycle low price data
  cl_price = record["metrics"]["cycle_low"]["price"]
  cl_date = record["metrics"]["cycle_low"]["at"]

  # Get current mcap
  mcap = record["metrics"]["marketcap"]["current_marketcap_usd"]
  liquid_mcap = record["metrics"]["marketcap"]["liquid_marketcap_usd"]

  # Get current price data
  current_price = record["metrics"]["market_data"]["price_usd"]

  # Get ROI data
  pct_changes_keys = ["percent_change_last_1_year", "percent_change_year_to_date", "percent_change_last_3_months", "percent_change_last_1_month", "percent_change_month_to_date", "percent_change_last_1_week",]
  pct_changes = {k:v for k,v in record["metrics"]["roi_data"].items() if k in pct_changes_keys}

  return dict(
      symbol = record["symbol"], 
      name = record["name"], 
      link = "https://messari.io/asset/{}".format(record["slug"]),
      current_price = current_price,
      circ_supply = circ_supply, 
      liquid_supply = liquid_supply, 
      ath_price = ath_price, 
      ath_date = ath_date, 
      cl_price = cl_price, 
      cl_date = cl_date, 
      mcap = mcap, 
      liquid_mcap = liquid_mcap,
      **pct_changes       
  )


# Get private fundraising data for coins (seed)

In [ ]:
# Extract initial private fundraising data -->  profile: investors>organizations, economics>launch>fundraising
# Using pandas
def get_fundraise_data(record, private_sales_rounds_titles_clean=private_sales_rounds_titles_clean):
  # Init
  investors = []
  first_private_sale_date = np.nan
  last_private_sale_date = np.nan
  num_tokens_private_sale = np.nan
  tot_usd_raised_private_sale = np.nan
  min_price_private_sale = np.nan
  max_price_private_sale = np.nan
  wavg_price_private_sale = np.nan 
  init_supply = np.nan
  fdv_private_sale = np.nan
  asset_collected = np.nan

  # Get total token supply
  init_supply = record["profile"]["economics"]["launch"]["initial_distribution"]["initial_supply"]

  # Get list of private investors
  coin_investors = record["profile"]["investors"]["organizations"] 
  if coin_investors != None:
    investors = [i["name"] for i in coin_investors]

  # Get details on sales rounds
  sales_rounds = record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]
  if sales_rounds:
    df = pd.DataFrame.from_records(sales_rounds)

    # Filter records of private token sales
    df1 = df[df.title_clean.isin(private_sales_rounds_titles_clean)]
    # Get earliest & last date of private sales
    first_private_sale_date = df1.end_date.astype(str).min()
    last_private_sale_date = df1.end_date.astype(str).max()

    # Calc total number of tokens sold during private sales
    num_tokens_private_sale = df1.native_tokens_allocated.sum()
    # Calc total USD amount raised in private sales
    tot_usd_raised_private_sale = df1.amount_collected_in_usd.sum()

    # Get min/max private sale token price 
    min_price_private_sale = df1.equivalent_price_per_token_in_usd.min()
    max_price_private_sale = df1.equivalent_price_per_token_in_usd.max()

    # Calc weighted avg token price of private sales
    if num_tokens_private_sale:
      wavg_price_private_sale = sum(df1.equivalent_price_per_token_in_usd * df1.native_tokens_allocated) / num_tokens_private_sale

    # Calc FDV based on last private sale
    if init_supply:
      fdv_private_sale = max_price_private_sale * init_supply

    # Safety check: currency collected in private fundraise
    asset_collected = df1.asset_collected.unique().tolist()

  return dict(
        investors = investors, 
        first_private_sale_date = first_private_sale_date, 
        last_private_sale_date = last_private_sale_date, 
        num_tokens_private_sale = num_tokens_private_sale, 
        tot_usd_raised_private_sale = tot_usd_raised_private_sale, 
        min_price_private_sale = min_price_private_sale, 
        max_price_private_sale = max_price_private_sale, 
        wavg_price_private_sale = wavg_price_private_sale, 
        init_supply = init_supply, 
        fdv_private_sale = fdv_private_sale,
        asset_collected = asset_collected,
    )


# Generate tables

## Current Market Data Table

In [10]:
# Make df with current market data (price, mcap, circ supply, first trade, max supply, status) 
mkt_data = [get_market_data(record) for record in data]
df_mkt_data = pd.DataFrame.from_records(mkt_data)
# Clean symbols, drop dups
df_mkt_data.symbol =  df_mkt_data.symbol.str.replace("$","").replace("-","").replace(".","").replace(" ","").str.upper().astype(str)
nomics_df = nomics_df.drop_duplicates("symbol")
print(df_mkt_data.shape)
print(nomics_df.shape)

# Join dfs: Add first trade, max supply, status from nomics
df_mkt_data1 = df_mkt_data.merge(nomics_df, how="left", left_on="symbol", right_on="symbol")
print(df_mkt_data1.shape)

# Current market data set 
numeric_cols = ["market_cap", "circulating_supply", "max_supply"]
df_mkt_data1[numeric_cols] = df_mkt_data1[numeric_cols].astype(float)
df_mkt_data1["fdv_current"] = df_mkt_data1.max_supply * df_mkt_data1.current_price

(2000, 18)
(1707, 6)
(2000, 23)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


## Valuations at fundraising event vs current valuation

In [12]:
# Valuations at fundraising event vs current valuation (mcap, fdv)
print(fdf.shape)
print(df_mkt_data1.shape)

fvals = fdf.merge(df_mkt_data1, how="left", left_on="symbol", right_on="symbol")
print(fvals.shape)

# Calc return / valuation delta since fundraise event 
fvals["fdv_at_raise"] = fvals.max_supply * fvals.equivalent_price_per_token_in_usd
fvals["val_current_delta"] = fvals.fdv_current / fvals.fdv_at_raise - 1

fvals1 = fvals[['symbol', 'title_clean', 'type', 'end_date', 'equivalent_price_per_token_in_usd', 'cum_amount_collected_in_usd','current_price',
       'circulating_supply', 'max_supply', 'fdv_current', 'fdv_at_raise', 'val_current_delta']].round(2)
fvals1

(362, 10)
(2000, 24)
(384, 33)


,symbol,title_clean,type,end_date,equivalent_price_per_token_in_usd,cum_amount_collected_in_usd,current_price,circulating_supply,max_supply,fdv_current,fdv_at_raise,val_current_delta
0,ETH,token sale,public,2014-09-02T16:00:00Z,0.31,18300000.0,1645.79,121818078.0,NaN,NaN,NaN,NaN
1,BNB,initial coin offering (ico),public,2017-07-21T04:00:00Z,0.15,15000000.0,303.16,163276975.0,1.632770e+08,4.949905e+10,2.449155e+07,2020.07
2,SOL,seed sale,private,2018-04-05T00:00:00Z,0.04,3170000.0,39.70,346652504.0,5.081810e+08,2.017413e+10,2.032724e+07,991.47
3,SOL,founding sale,series a,2018-06-03T00:00:00Z,0.20,15800000.0,39.70,346652504.0,5.081810e+08,2.017413e+10,1.016362e+08,197.49
4,SOL,validator sale,series a,2019-07-09T00:00:00Z,0.22,21500000.0,39.70,346652504.0,5.081810e+08,2.017413e+10,1.143407e+08,175.44
...,...,...,...,...,...,...,...,...,...,...,...,...
379,QI,public sale a,public,2021-04-28T00:00:00Z,0.01,9522000.0,0.02,NaN,NaN,NaN,NaN,NaN
380,QI,public sale b,public,2021-04-28T00:00:00Z,0.01,10105200.0,0.02,NaN,NaN,NaN,NaN,NaN
381,QI,private sale,private,2021-04-28T00:00:00Z,0.01,6228000.0,0.02,NaN,NaN,NaN,NaN,NaN
382,ASK,saft (saftlaunch),public,2018-03-02T05:00:00Z,0.00,1287431.0,NaN,NaN,1.000000e+12,NaN,7.200000e+08,NaN


## Last Private Fundraise vs Current Valuation

In [13]:
private_rounds = ['private',]

last_fdf = fvals1[fvals1["type"].isin(private_rounds)].sort_values('end_date',ascending=True).groupby("symbol").last()
print(last_fdf.shape)

last_fdf

(92, 11)


,title_clean,type,end_date,equivalent_price_per_token_in_usd,cum_amount_collected_in_usd,current_price,circulating_supply,max_supply,fdv_current,fdv_at_raise,val_current_delta
symbol,,,,,,,,,,,
AION,token sale (usd),private,2017-09-25T16:00:00Z,0.50,15624557.59,0.04,5.084321e+08,5.084321e+08,2.129746e+07,2.542160e+08,-0.92
AKRO,initial coin offering (ico),private,2018-07-01T03:59:00Z,NaN,NaN,0.01,1.000000e+08,5.000000e+09,2.560682e+07,NaN,NaN
AKT,private sale,private,2020-03-30T00:00:00Z,0.06,2000000.00,0.39,NaN,NaN,NaN,NaN,NaN
ALPHA,private presale,private,None,0.02,4000000.00,0.03,NaN,NaN,NaN,NaN,NaN
AMP,private sale (fxc),private,2018-12-01T00:00:00Z,0.00,14100000.00,0.01,1.006056e+08,6.224108e+08,5.185019e+06,6.224108e+05,7.33
...,...,...,...,...,...,...,...,...,...,...,...
XLM,seed round,private,2014-05-31T04:00:00Z,0.00,3000000.00,0.12,2.517047e+10,5.000179e+10,5.825324e+09,7.500268e+07,76.67
YGG,series a (b tranche),private,2021-06-01T00:00:00Z,0.09,3325000.00,0.80,NaN,NaN,NaN,NaN,NaN
ZEC,venture funding,private,2016-03-31T04:00:00Z,15.24,3000000.00,63.17,1.271966e+07,2.100000e+07,1.326492e+09,3.200400e+08,3.14


## All investors (Messari DB) most held coins 

In [14]:
inv_count_df = inv_df1.groupby("symbol").count().sort_values("investors", ascending=False)[["name"]]
print(inv_count_df.shape)
inv_count_df1 = inv_count_df.merge(inv_df.set_index("symbol")[["investors"]], left_index=True, right_index=True)
print(inv_count_df1.shape)

inv_count_df1[inv_count_df1.name > 1].sort_values("name", ascending=False)

(1903, 1)
(1903, 2)


,name,investors
symbol,,
ZIL,57,"[Global Brain, DU Capital, Wolfedge Capital, D..."
PERL,48,"[Reciprocal Ventures, ZhenFund, Link Ventures,..."
MFT,42,"[Force Over Mass Capital, One Block Capital, S..."
DOT,41,"[KR1, zk Capital, Kosmos Capital, BlockAsset V..."
OGN,39,"[One Block Capital, BlockTower Capital, 122WES..."
...,...,...
CREDI,2,"[AU21 Capital, X21 Capital]"
ASK,2,"[Collinstar Capital, Vendetta Capital]"
PIVX,2,"[Myriad Capital Management, Parallax Digital]"


## Top Investors most held coins

In [32]:
top_vcs = [
  "Alameda Research" 
  'a16z Crypto',
  'Andreessen Horowitz',
  'Three Arrows Capital',
  'Coinbase Ventures',
  'Y Combinator',
  'FTX ',
  'CMS Holdings',
  'Delphi Digital',
  'Founders Fund',
  'Draper Associates',
  'Electric Capital',
  'Wintermute',
  'Dragonfly Capital',
  'Sequoia Capital',
  'Pantera Capital',
  'Multicoin Capital',
  'Binance Labs',
  'Paradigm',
  'Tiger Global',
  'Union Square Ventures',
  'Jump Capital',
  'Polychain Capital',
  'Winklevoss Capital Management, LLC',
  'Zee Prime Capital',
  'eGirl Capital',
  'zk Capital'
]

top_inv_df1 = inv_df1[inv_df1.investors.isin(top_vcs)]
top_inv_df = top_inv_df1.groupby("symbol").investors.agg(list).reset_index()

inv_count_df2 = top_inv_df1.groupby("symbol").count().sort_values("investors", ascending=False)[["name"]]
inv_count_df3 = inv_count_df2.merge(top_inv_df.set_index("symbol")[["investors"]], left_index=True, right_index=True)

In [33]:
inv_count_df3

,name,investors
symbol,,
BTC,10,"[Winklevoss Capital Management, LLC, Pantera C..."
ETH,10,"[Electric Capital, Pantera Capital, Three Arro..."
DYDX,9,"[Dragonfly Capital, Three Arrows Capital, CMS ..."
MINA,9,"[Paradigm, Pantera Capital, Polychain Capital,..."
CKB,7,"[Sequoia Capital, CMS Holdings, Multicoin Capi..."
...,...,...
LBA,1,[Binance Labs]
KDA,1,[Multicoin Capital]
KAVA,1,[Binance Labs]


symbol
1INCH    [Pantera Capital, Binance Labs, FTX , Dragonfl...
AAVE     [Three Arrows Capital, Pantera Capital, Multic...
ACH                                      [Pantera Capital]
ADX                                    [Zee Prime Capital]
ALCX     [Binance Labs, eGirl Capital, CMS Holdings, De...
                               ...                        
YFI                 [Three Arrows Capital, Delphi Digital]
YGG                                       [Delphi Digital]
ZEC      [Pantera Capital, Electric Capital, Winklevoss...
ZIL                                    [Polychain Capital]
ZRX      [Coinbase Ventures, Pantera Capital, Polychain...
Name: investors, Length: 174, dtype: object